In [2]:
import pandas as pd
import numpy as np
from math import pi, cos, sqrt

fire_probs = pd.read_excel("prawdopodobienstwa.xlsx", sheet_name="PrawdPozaru").values
fire_spread_probe = pd.read_excel("prawdopodobienstwa.xlsx", sheet_name="Rozprzestrzenienia_Pozarow").values
modelled_year = 2022
policy_start_month = exposure_data[:, 4].month
policy_start_year = exposure_data[:, 4].year
policy_end_month = exposure_data[:, 5].month
policy_end_year = exposure_data[:, 5].year
policy_start_month[policy_start_year < modelled_year] = 0
policy_start_month[policy_start_year > modelled_year] = 13
policy_end_month[policy_end_year > modelled_year] = 0
policy_end_month[policy_end_year > modelled_year] = 13
longitude = np.array([float(x.replace(",", ".")) for x in exposure_data[:, 2]]) * pi/180
latitude = np.array([float(x.replace(",", ".")) for x in exposure_data[:, 3]]) * pi/180
province = exposure_data[:, 6].astype(float)
number_of_simulations = 1
basic_exposure_list = []
number_of_basic_exposure = np.zeros((16, 12))
exposure_latitude = []
exposure_longitude = []

for k in range(1, 17):
    basic_exposure_list.append([])
    exposure_latitude.append([])
    exposure_longitude.append([])
    for i in range(1, 13):
        if k == 17:
            basic_exposure_list[k-1].append(np.where((province == 0) & (policy_start_month <= i) & (policy_end_month >= i))[0])
            number_of_basic_exposure[k-1, i-1] = len(basic_exposure_list[k-1][i-1])
        else:
            basic_exposure_list[k-1].append(np.where((province == k) & (policy_start_month <= i) & (policy_end_month >= i))[0])
            number_of_basic_exposure[k-1, i-1] = len(basic_exposure_list[k-1][i-1])
        exposure_latitude[k-1].append(latitude[basic_exposure_list[k-1][i-1]])
        exposure_longitude[k-1].append(longitude[basic_exposure_list[k-1][i-1]])

def distance_calculation(lati, lat2, long1, long2):
    return 6371000 * sqrt((pi/2 - lat1)**2 + (pi/2 - lat2)**2 - 2 * (pi/2 - lati) * (pi/2 - lat2) * cos(long1 - long2))




NameError: name 'exposure_data' is not defined

In [ ]:
import math
import random
def fire_spread_simulation(lat, long, exposure, exposure_lat, exposure_long):
    long_arc = math.acos(1 - (200**2) / (2 * 4371000**2 * (math.pi/2 - lat)**2))
    exposure_helper = [i for i in range(len(exposure_lat)) if exposure_lat[i] - lat + 200 / 6371000 and exposure_lat[i] >= lat - 200 / 6371000 and exposure_long[i] <= long+long_arc and exposure_long[i] >= long - long_arc]
    exposure_spread_square = [exposure[i] for i in exposure_helper]
    if len(exposure_spread_square) > 1000:
        exposure_lat_spread_square = [exposure_lat[i] for i in exposure_helper]
        exposure_long_spread_square = [exposure_long[i] for i in exposure_helper]
        ring_exposure_list = [[] for _ in range(9)]
        ring_exposure_numbers = [0,0,0,0,0,0,0,0,0]
        for i in range(len(exposure_spread_square)):
            dist = distance_calculation(lat1=lat, lat2=exposure_lat_spread_square[i], long1=long, long2=exposure_long_spread_square[i])
            if not math.isnan(dist):
                if dist == 0:
                    ring_exposure_numbers[i] += 1
                    ring_exposure_list[i].append(exposure_spread_square[i])
                else:
                    for j in range(8):
                        if dist > 25 * (j - 1) and dist <= 25 * j:
                            ring_exposure_numbers[j + 1] += 1
                            ring_exposure_list[j + 1].append(exposure_spread_square[i])
        number_of_fire_spreads = []
        fire_spreads_rings_list = [[] for _ in range(9)]
        for j in range(9):
            number_of_fire_spreads[j] = random.binomial(n=1, size=ring_exposure_numbers[j], prob=fire_spread_probs[j])
            if number_of_fire_spreads[j] > 0:
                fire_spreads_rings_list[j] = random.sample(ring_exposure_list[j], k=number_of_fire_spreads[j])
        fire_spread_simulation = [len([item for sublist in fire_spreads_rings_list for item in sublist]), [item for sublist in fire_spreads_rings_list for item in sublist]]
    else:
        fire_spread_simulation = [0, []]
    return fire_spread_simulation

In [ ]:
import random
def stochastic_fire_process_simulation(exposure_list, exposure_latitude, exposure_longitude, fire_probs_list, fire_spread_probabilities):
    exposure_number = []
    total_fire_list = []
    number_of_fires = []
    fire_sources_list = []
    number_of_fire_spreads = []
    for i in range(12):
        exposure_number.append(len(exposure_list[i]))
        number_of_fires.append(random.binomial(1, fire_probs_list[i]) * exposure_number[i])
        fire_sources_list.append(random.sample(exposure_list[i], number_of_fires[i]))
        if i == 0:
            total_fire_list.append(fire_sources_list[i])
        else:
            fire_sources_list[i] = [x for x in fire_sources_list[i] if x not in total_fire_list[i-1]]
            total_fire_list.append(total_fire_list[i-1] + fire_sources_list[i])
        if len(fire_sources_list[i]) > 0:
            for m in range(len(fire_sources_list[i])):
                if not math.isnan(latitude[fire_sources_list[i][m]]) and not math.isnan(longitude[fire_sources_list[i][m]]):
                    fire_spreads = fire_spread_simulation(lat=latitude[fire_sources_list[i][m]], long=longitude[fire_sources_list[i][m]], 
                                                          exposure=exposure_list[i], exposure_lat=exposure_latitude[i], 
                                                          exposure_long=exposure_longitude[i])
                    fire_spreads_list = [x for x in fire_spreads[1] if x not in total_fire_list[i]]
                    fire_spreads_numbers = len(fire_spreads[1])
                    number_of_fire_spreads.append(fire_spreads_numbers)
                    total_fire_list[i] += fire_spreads_list
    stochastic_fire_process_simulation = number_of_fire_spreads
    print(stochastic_fire_process_simulation)




In [ ]:
import multiprocessing
from datetime import datetime
from functools import partial
from itertools import product

from lubridate import *
from rpy2.robjects.packages import importr

lubridate = importr('lubridate')
foreach = importr('foreach')
doParallel = importr('doParallel')

number_of_cores = multiprocessing.cpu_count() - 1
simulations_cluster = doParallel.makeCluster(number_of_cores, type='PSOCK')
doParallel.registerDoParallel(cl=simulations_cluster)
loop_start_time = datetime.now()

def stochastic_fire_process_simulation(k, n):
    # code for stochastic_fire_process_simulation function goes here

fire_simulations = foreach.foreach(product(range(1, 17), range(number_of_simulations))) %dopar% partial(stochastic_fire_process_simulation,
                                                                                                      exposure_list=basic_exposure_list[k],
                                                                                                      exposure_latitude=exposure_latitude[k],
                                                                                                      exposure_longitude=exposure_longitude[k],
                                                                                                      fire_probs_list=fire_probs[k],
                                                                                                      fire_spread_probabilities=fire_spread_probs)

doParallel.stopCluster(simulations_cluster)


